<a href="https://colab.research.google.com/github/AndreChirva/Spark-SQL/blob/main/Top_10_%D1%81%D1%82%D1%80%D0%B0%D0%BD_%D1%81_%D0%BC%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%BC_%D0%B7%D0%B0%D1%84%D0%B8%D0%BA%D1%81%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC_%D0%BA%D0%BE%D0%BB_%D0%B2%D0%BE%D0%BC_%D0%BD%D0%BE%D0%B2%D1%8B%D1%85_%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0%B5%D0%B2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, max, row_number, desc
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [121]:
spark = SparkSession.builder.appName("covid_analysis").getOrCreate()

In [ ]:
df = spark.read.csv("covid-data.csv", header = True)
df.show()

In [ ]:
result_df = (df.filter(F.col("date").between("2021-03-25", "2021-03-31"))
               .withColumn("Число", F.date_format(col("date"), "dd"))
               .withColumn("Страны", F.col("location"))
               .withColumn("кол-во новых случаев", F.col("new_cases")))
result_df = (result_df.groupBy("Страны","Число")
                      .agg(F.sum("кол-во новых случаев").alias("Кол-во новых случаев")))
windowSpec = Window.partitionBy("Страны").orderBy(col("Кол-во новых случаев").desc())
result_df = result_df.withColumn("number", F.row_number().over(windowSpec))
result_df.show()

In [137]:
result = (result_df.filter(F.col("number") == 1)
                   .filter(~F.col("Страны").isin("World", "Europe", "European Union", "Asia"))
                   .select("Число", "Страны", "Кол-во новых случаев")
                   .orderBy(F.col("Кол-во новых случаев").desc())
                   .limit(10))

result.show()


+-----+-------------+--------------------+
|Число|       Страны|Кол-во новых случаев|
+-----+-------------+--------------------+
|   25|South America|            148476.0|
|   25|       Brazil|            100158.0|
|   26|North America|             93350.0|
|   26|United States|             77321.0|
|   31|        India|             72330.0|
|   31|       France|             59054.0|
|   31|       Turkey|             39302.0|
|   26|       Poland|             35145.0|
|   31|      Germany|             25014.0|
|   26|        Italy|             24076.0|
+-----+-------------+--------------------+

